<a href="https://colab.research.google.com/github/OmniaOsama03/Artificial_Intelligence_Project_2/blob/main/Fine-tuning_Academic_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.7/397.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 14.7 MB/s eta 0:00:00


In [ ]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "cVc33fsF5XVKcNVaEpgDfpFbKLfsQX5I"
os.environ['GRADIENT_WORKSPACE_ID'] = "ba1a0084-82f6-42d1-befd-a46d7b1ebc83_workspace"

In [ ]:
!pip install gradio==3.48.0
import gradio as gr
from gradientai import Gradient


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 28.9 M

#Model Deployment

The Model before text preprocessing & Fine-tuning (BASE MODEL)

In [ ]:
def CreateBaseModel():
    with Gradient() as gradient:
        # Get the base model
        base_model = gradient.get_base_model(base_model_slug = "nous-hermes2")

        def get_base_model_response(userQuery):
            MyOutput = base_model.complete(query = userQuery).generated_output
            return MyOutput

        gr.Interface(fn=get_base_model_response,
                     outputs="text",
                     title="Base Model Chatbot",
                     inputs="text",
                     description="Interact with the base model. Try asking something!").launch()

CreateBaseModel()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://c11f4e444e09993aae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Fine-tuned Model **before** preprocessing

In [ ]:
def CreateBaseModel():
    with Gradient() as gradient:
        base_model = gradient.get_base_model(base_model_slug="nous-hermes2")
        GRP2_newModel = base_model.create_model_adapter(name="test model")

        print(f"Created model adapter with id {GRP2_newModel.id}")

        MyPrompts=([

    {"inputs": "### Instruction: What are administrative drops, and under what circumstances are they initiated in Abu Dhabi University?\n\n### Response: Administrative drops at Abu Dhabi University are initiated by university officials for various reasons, including failure to meet prerequisites, safety concerns, academic or disciplinary suspension, disruptive behavior, or exceeding allowable absences or course loads. These drops ensure academic integrity and the safety of students and faculty members. It's crucial for students to be aware of these regulations and adhere to them to maintain their academic standing."},
    {"inputs": "### Instruction: What is the maximum number of courses I can take in a major semester in Abu Dhabi University?\n\n### Response: The maximum number of courses you can take in a major semester is 6 courses, accumulating a maximum of 18 credit hours."},
    {"inputs": "### Instruction: What circumstances may result in a change of a student's final grade at Abu Dhabi University?\n\n### Response: There are several situations that may lead to a change in a student's final grade:\n\n- A grade appeal request by the student after an informal discussion with the faculty member.\n- Discovery of an error in calculating the student's final grade.\n- Change of an \"I\" (Incomplete) or \"IP\" (In Progress) grade to the actual grade upon release. It's crucial for students to be aware of these possibilities and the time limit for grade changes, which is one semester from the date the grades are posted by the Registrar."},
    {"inputs": "### Instruction: What does an advisor do in Abu Dhabi University?\n\n### Response: An advisor is a full-time staff member within each college who advises and counsels students on programs and course selection, institutional policies, career choices, effective study habits, and other academic and career-oriented decisions."},
    {"inputs": "### Instruction: What is the process for appealing a final grade at Abu Dhabi University?\n\n### Response: If a student wishes to appeal a final grade, they must follow the Grade Appeal Procedure outlined by the university. This involves a consultation process with the faculty member, department chairperson, and dean of the college offering the course. If the matter is not resolved, the student may initiate a Committee Appeal Process by filing a Grade Appeal Form with the Office of the Registrar. The committee reviews the student's performance and makes recommendations, with the final decision resting with the dean. It's important for students to adhere to the deadlines and procedures outlined for grade appeals."},
    {"inputs": "### Instruction: When is the add and drop period in Abu Dhabi University?\n\n### Response: A student is allowed to add/drop one or more courses during the first week of the regular semester and during the first two days of the Winter/Summer term."},
    {"inputs": "### Instruction: How are students classified based on their progression towards their Bachelor Degree at Abu Dhabi University?\n\n### Response: Each college at Abu Dhabi University has its own classification system based on the number of credit hours passed, tailored to the specific requirements of their programs. This classification helps students track their progress towards completing their Bachelor's Degree effectively:\n\n- College of Arts (CAS):\n  - First Year/Freshmen: 00 - 29 credit hours\n  - Second Year/Sophomores: 30 - 59 credit hours\n  - Third Year/Juniors: 60 - 89 credit hours\n  - Final Year/Seniors: 90+ credit hours\n\n- College of Business (COB):\n  - First Year/Freshmen: 00 - 30 credit hours\n  - Second Year/Sophomores: 31 - 60 credit hours\n  - Third Year/Juniors: 61 - 90 credit hours\n  - Final Year/Seniors: 91+ credit hours\n\n- College of Engineering (COE):\n  - Bachelor of Architecture\n    - First Year: 00 - 34 credit hours\n    - Second Year: 35 - 69 credit hours\n    - Third Year: 70 - 102 credit hours\n    - Fourth Year: 103 -132 credit hours\n    - Fifth Year: 133+ credit hours\n  - B.Sc. in Aviation, Biomedical Engineering, Chemical Engineering, Civil Engineering, Computer Engineering, Cybersecurity Engineering, Electrical Engineering, Industrial Engineering, Information Technology, Interior Design, Mechanical Engineering, and Software Engineering\n    - First Year/Freshmen: 00 - Various credit hours\n    - Second Year/Sophomores: Various - Various credit hours\n    - Third Year/Juniors: Various - Various credit hours\n    - Final Year/Seniors: Various+ credit hours\n\n- College of Health Sciences (CoHS):\n  - B.Sc. in Public Health, Environmental Health and Safety, Biomedical Sciences - Laboratory Medicine, Molecular and Medical Genetics, and Human Nutrition and Dietetics\n    - First Year/Freshmen: 00 - Various credit hours\n    - Second Year/Sophomores: Various - Various credit hours\n    - Third Year/Juniors: Various - Various credit hours\n    - Final Year/Seniors: Various+ credit hours"},
    {"inputs": "### Instruction: Can I drop courses after the add and drop period ends during the Spring/Fall semester in Abu Dhabi University?\n\n### Response: Students dropping courses within the first calendar week of the Fall/Spring semester will receive a 100% refund of the tuition fee. Students dropping courses in the second calendar week of the Fall/Spring semester will receive 75% refund of the tuition fee. In such cases, a “Withdrawal without Penalty” (W) grade will be entered in their records. Students dropping courses in the third calendar week of the Fall/Spring semester will receive a 50% refund of tuition fees. In such cases, a (W) grade will be entered in their records. Students dropping courses after the third week of the Fall/Spring semester will receive no refund, and will be awarded a (W) grade for that course. If students do not withdraw from courses during these specified periods, they will be considered as being registered for the courses and be held accountable."},
    {"inputs": "### Instruction: What are the consequences of violating the Academic Integrity Policy (AIP) at Abu Dhabi University?\n\n### Response: Violating the Academic Integrity Policy (AIP) at Abu Dhabi University can lead to severe consequences, including dismissal from the university. Examples of violations include cheating, plagiarism, fabrication of data, presenting false credentials, collusion, and free riding. Sanctions may include failing grades, course withdrawals, and ineligibility for tuition refunds."},
    {"inputs": "### Instruction: How can I apply for graduation in Abu Dhabi University?\n\n### Response: To apply for graduation, visit www.adu.ac.ae and log in to PeopleSoft using your username and password. Then, navigate to self-service and select 'degree progress/graduation.' Proceed to apply for graduation, choose your program, select the expected graduation term, and carefully read any instructions provided by the Office of the Registrar."},
    {"inputs": "### Instruction: What is the total credit hour requirement for the Cybersecurity Engineering program, and how is it distributed across different course categories in Abu Dhabi University?\n\n### Response: The Cybersecurity Engineering program at Abu Dhabi University requires 135 credit hours for completion. These are distributed across various categories, including general education (24 credit hours), college requirements (54 credit hours), major requirements (42 credit hours), major electives (6 credit hours), and open electives (3 credit hours)."},
    {"inputs": "### Instruction: How is the grading scale at ADU and what does it mean?\n\n### Response: At Abu Dhabi University, the grading scale ranges from A (4.00) to F (0.00). Each grade corresponds to a specific range of percentages and carries different meanings, such as excellent, very good, good, satisfactory, or fail. Additionally, there are other symbols like NP (Not Pass), SP (Special Pass), P (Pass), S (Successful Completion), U (Unsuccessful Completion), I (Incomplete), IP (In Progress), T (Transfer), H (Final Grade on Hold), W (Withdrawal from a Course), and WA (Withdrawal Due to Absence Limit)."},
    {"inputs": "### Instruction: What are the major requirements for the Bachelor of Science in Information Technology program at Abu Dhabi University?\n\n### Response: The major requirements for the Bachelor of Science in Information Technology program consist of 45 credit hours. These include courses such as Ethical Hacking, Data Warehousing and Data Mining, Web Design, Operating Systems, Artificial Intelligence, Software Engineering, and others. Each course has its prerequisites and contributes to building a comprehensive understanding of information technology."},
    {"inputs": "### Instruction: What are the major requirements for the Bachelor of Science in Cybersecurity Engineering program at Abu Dhabi University?\n\n### Response: The major requirements for the Bachelor of Science in Cybersecurity Engineering program include 42 credit hours. These courses cover essential topics such as Database Management Systems, Digital Logic Design, Web Design, Artificial Intelligence, Information Security, Internet of Things, Network Security, and Ethical Hacking. Prerequisites are specified for each course."},
    {"inputs": "### Instruction: Am I eligible to take my capstone next semester at Abu Dhabi University?\n\n### Response: To determine your eligibility for the capstone next semester, ensure you have reached the senior level (90 credit hours) and completed any prerequisites. The capstone typically requires a certain level of academic progress and completion of foundational coursework."},
    {"inputs": "### Instruction: Tell me about the Information Technology degree in Abu Dhabi University?\n\n### Response: The Information Technology (IT) program at Abu Dhabi University offers concentrations in Web Technology and Internet Applications, Networking and Security, and Mobile Applications and Interactive Media. These concentrations provide specialized knowledge and skills in areas such as web development, cybersecurity, mobile app development, and user interface design."},
    {"inputs": "### Instruction: How should I plan my course load for the next semester in Abu Dhabi University?\n\n### Response: When planning your course load for the next semester, consider the types of courses you intend to take and your ability to manage workload. It's advisable to balance major courses with electives and to avoid overloading yourself, especially if you anticipate difficulty with certain subjects."},
    {"inputs": "### Instruction: What are the major requirements for the Bachelor of Science in Software Engineering program at Abu Dhabi University?\n\n### Response: The major requirements for the Bachelor of Science in Software Engineering program consist of 36 credit hours. Courses cover topics such as Database Management Systems, Web Design, Artificial Intelligence, Distributed Information Systems, Information Security, and various software engineering principles."},
    {"inputs": "### Instruction: What are the major elective courses available in the Bachelor of Science in Software Engineering program at Abu Dhabi University?\n\n### Response: Major elective courses for the Bachelor of Science in Software Engineering program offer options such as Internet of Things, Data Science, Web Programming, E-Commerce Applications Design, and Selected Topics in Software Engineering. These courses allow students to tailor their education to specific interests within the field."},
    {"inputs": "### Instruction: What are the major elective courses available in the Bachelor of Science in Cybersecurity Engineering program at Abu Dhabi University?\n\n### Response: Major elective courses for the Bachelor of Science in Cybersecurity Engineering program include options like Data Science, System and Network Administration, Selected Topics in Cybersecurity, and various software engineering courses. These electives provide students with opportunities to specialize further in areas of interest."},
    {"inputs": "### Instruction: What are the major elective courses available in the Information Technology program at Abu Dhabi University?\n\n### Response: Major elective courses for the Information Technology program include options such as Computer Graphics and Animation, Mobile Device Security, Network Security and Forensics, Web Programming, and Advanced E-commerce Applications Design. These courses offer additional depth and specialization within the IT field."},
    {"inputs": "### Instruction: What concentrations are offered in the College of Engineering at Abu Dhabi University?\n\n### Response: The College of Engineering at Abu Dhabi University offers concentrations such as Artificial Intelligence in Computer Engineering, Robotics and Automation in Electrical Engineering, Industrial Mechatronics and Metallurgy in Mechanical Engineering, Water Technology in Chemical Engineering, and Cybersecurity and Game Development in Information Technology, among others."},
    {"inputs": "### Instruction: What are the requirements to enroll in the Engineering and Computer Science college Majors in Abu Dhabi University?\n\n### Response: To enroll in majors within the College of Engineering and Computer Science at Abu Dhabi University, you need to have completed a minimum of 24 semester credit hours of university requirements with a GPA of at least 2.25. Additionally, specific courses such as ITE 100, ENG 105, and MTT 101 must be completed with a grade of at least 2.0."},
    {"inputs": "### Instruction: How do I know which courses I should take next semester?\n\n### Response: To determine which courses to take next semester, refer to your study plan and identify the courses available in that semester. Some courses may only be offered in specific semesters, so it's essential to plan accordingly based on your program requirements and academic progress."},
    {"inputs": "### Instruction: Can I take free elective courses from any Major in the university?\n\n### Response: Yes, you can take free elective courses from any Major in the university, provided you meet the prerequisites. However, for the BEd Field Teaching (Arabic Language and Islamic Studies) curriculum, only 2 specific courses are allowed as electives."},
    {"inputs": "### Instruction: Do I have to complete a Minor?\n\n### Response: No, completing a Minor is optional but strongly encouraged. Minors consist of 6 courses (18 credits), which can be substituted for 6 elective courses in some Majors."},
    {"inputs": "### Instruction: What Majors and Minors are available?\n\n### Response: The list of Majors and minors is available in the Undergraduate/Postgraduate Catalog and the Advising website on the ADU portal. Minors offered are identical to the Majors available."},
    {"inputs": "### Instruction: When should I declare my Major?\n\n### Response: Decide your Major when you're close to completing the 14 University Requirement courses, as you'll then begin taking courses from your Major."},
    {"inputs": "### Instruction: What are the admission requirements for direct entry into the Information Technology program?\n\n### Response: Direct entry requires a minimum average of 65% for Elite Track, 70% for Advanced Track, and 75% for General Track or equivalent. EmSAT scores in Mathematics, Physics, and English Language are also required."},
    {"inputs": "### Instruction: How many undergraduate programs does the College of Engineering  offer, and what are they?\n\n### Response: The College offers thirteen undergraduate programs covering Mechanical, Industrial, Electrical, Computer, Biomedical, Cybersecurity, Software, Information Technology, Civil, Chemical, Architecture, Interior Design, and Aviation."},
    {"inputs": "### Instruction: What are the different scholarships the university offers?\n\n### Response: Scholarships include H.H. Sheikh Hamdan Bin Zayed Scholarship, Chairman’s Scholarship, University Scholarship, Academic Scholarship, and Athletic Scholarship."},
    {"inputs": "### Instruction: How can I achieve Latin honors at ADU?\n\n### Response: To achieve Latin honors, maintain a Cumulative Grade Point Average (CGPA) of 3.5 or above upon graduation, resulting in titles like Cum laude (3.50-3.69), Magna cum laude (3.70-3.89), or Summa cum laude (3.90-4.00)."},
    {"inputs": "### Instruction: What are the requirements to apply for financial aid?\n\n### Response: Financial aid is determined based on eligibility and need, ranging from 10% to 30% tuition fee waiver. Initial requirements include UAE permanent residency, financial need evidence, and meeting English proficiency standards."},
    {"inputs": "### Instruction: If I Withdraw From The University, Can I Be Re-Admitted?\n\n### Response: Former students in good academic standing, who withdrew for reasons like financial issues or medical conditions, may seek re-admission. They must submit a petition to the Registrar at least two months before the semester begins."},
    {"inputs": "### Instruction: Can I Change My Major?\n\n### Response: Yes, you can change your Major at any time during the semester if in good standing. Simply fill out a Change Major Application Form at the college of the newly chosen major."},
    {"inputs": "### Instruction: What do I do if there is no more seats in a course I want to register in?\n\n### Response: You can talk to your advisor to either put you on a waiting list or talk to the department head to open another section. This can be done if a good number of students also need to take this course. If that's not possible, you will have to wait until the next semester/year."},
    {"inputs": "### Instruction: What is the Software Engineering degree about?\n\n### Response: The Bachelor of Science in Software Engineering program at Abu Dhabi University is designed to cultivate a new generation of software engineers equipped to tackle the multifaceted challenges of contemporary software development. Rooted in the application of engineering principles to the systematic creation of software, this program provides students with a comprehensive understanding of the theoretical underpinnings and practical applications essential for success in the field. Central to the program's mission is the development of technical proficiency, problem-solving abilities, and leadership qualities among its graduates. Through rigorous coursework and hands-on projects, students gain proficiency in key areas such as software maintenance and evolution, requirements engineering, software design and architecture, testing and quality assurance, web design, and more. By engaging with real-world challenges and industry-standard tools, students acquire the skills and confidence to navigate the complexities of modern software development projects. Moreover, the program fosters a culture of lifelong learning, recognizing the dynamic nature of the software industry and the importance of continuous skill development. Students are encouraged to pursue training, continuing education, and even graduate studies to stay abreast of emerging technologies and industry trends. By instilling a commitment to ongoing learning and professional growth, the program prepares graduates to adapt to evolving industry demands and thrive in their careers over the long term. Beyond technical skills, the program also places a strong emphasis on ethical values and social responsibility. Students are encouraged to consider the broader implications of their work on economic, environmental, social, political, and ethical dimensions. By upholding ethical standards and contributing to the development and growth of local and global communities, graduates of the program become not only proficient software engineers but also conscientious global citizens. In essence, the Bachelor of Science in Software Engineering program at Abu Dhabi University offers a holistic educational experience that prepares students to excel in the dynamic and ever-evolving field of software development. By fostering a blend of technical expertise, problem-solving abilities, lifelong learning habits, and ethical values, the program equips graduates with the tools they need to make meaningful contributions to both the software industry and society at large, positioning them for success in their careers and beyond."},
    {"inputs": "### Instruction: Can I drop courses after the add and drop period ends during the Winter/Summer semester?\n\n### Response: Students dropping courses within the first two days of the calendar days of the Winter/Summer semester will receive a 100% refund of the tuition fee. Students dropping courses in the third and fourth calendar days of the Winter/Summer semester will receive 75% refund of the tuition fee. In such cases, a â€œWithdrawal without Penaltyâ€ (W) grade will be entered in their records. Students dropping courses in the first and second days of the next calendar week of the Winter/Summer semester will receive 50% refund of tuition fees. In such cases, a (W) grade will be entered in their records. Students dropping courses after the two days of the second week will receive no refund, and will be awarded a (W) grade for that course. If students do not withdraw from courses during these specified periods, they will be considered as being registered for the courses and be held accountable. A 100% refund of tuition fees will be given for courses cancelled by Abu Dhabi University."},
    {"inputs": "### Instruction: What career paths are available with a degree in CSIST?\n\n### Response: There are multiple career paths in the computer science and information technology field. It all depends on your goals, aspirations, and preferences."},
    {"inputs": "### Instruction: What are the minimum admission requirements for direct entry into engineering programs?\n\n### Response: For direct admission to engineering programs, applicants must have a minimum high school average of 75% for the Elite Track or 80% for the Advanced Track, and 90% for the General Track or equivalent in standardized international systems. Additionally, they must pass EmSAT in Mathematics and Physics with scores of 800 or equivalent, and EmSAT in English Language with a score of 1100, or an approved equivalent English Proficiency test."},
    {"inputs": "### Instruction: How can I apply for internships?\n\n### Response: First, you need to make sure that you have completed the required number of credit hours. Then you can apply for internship on ADU groups once the portal opens. Then you need to start looking for companies or inform the department that you need help in looking for companies and they will guide you accordingly."},
    {"inputs": "### Instruction: What is the purpose of the grades P (credit), P (non-credit), S, and U?\n\n### Response: These grades indicate the completion status of certain academic activities: P (credit): Satisfactory completion of credit-bearing coursework P (non-credit): Satisfactory completion of non-credit ELI or undergraduate courses/Internship S: Satisfactory completion of graduate courses U: Unsatisfactory completion of graduate courses"},
    {"inputs": "### Instruction: What should I do if a required course is not available next semester?\n\n### Response: Inform your academic advisor then your department head so they can open the course."},
    {"inputs": "### Instruction: Do all students have to take a math placement test?\n\n### Response: Yes, all College of Engineering students must take the Math Placement Test (MPT) administered by the University College. Based on the MPT result, students will be placed in one of the math courses (MTH 100, MTT 101, or MTT 102) depending on their program of study."},
    {"inputs": "### Instruction: What do the grade notations NP, SP, P, S, U, I, IP, T, W, WA, and H stand for in Abu Dhabi University's grading system?\n\n### Response: These notations represent different aspects of a student's academic performance or status: NP: Not Pass SP: Special Pass P: Pass S: Successful Completion U: Unsuccessful Completion I: Incomplete IP: In Progress T: Transfer W: Withdrawal from a Course WA: Withdrawal Due to Absence Limit H: Final Grade on Hold"},
    {"inputs": "### Instruction: What additional admission requirement applies to applicants to the Bachelor of Architecture program?\n\n### Response: Applicants to the Bachelor of Architecture Program are required to submit or present a portfolio of graphic work for evaluation as part of the admission requirements. The portfolio should demonstrate creativity and/or artistic skill. Applicants may also be selectively interviewed by two members of the teaching staff to assess their genuine interest in the subject and evidence of creative intent."},
    {"inputs": "### Instruction: What is the grading scale used for undergraduate courses at Abu Dhabi University?\n\n### Response: Undergraduate courses at Abu Dhabi University use the following grading scale: A: 4.00 (90-100) Excellent B+: 3.50 (85-89) Very Good B: 3.00 (80-84) Very Good C+: 2.50 (75-79) Good C: 2.00 (70-74) Good D+: 1.50 (65-69) Satisfactory D: 1.00 (60-64) Satisfactory F: 0.00 (Less than 60) Fail"},
    {"inputs": "### Instruction: What is an independent study course, and who is eligible to participate?\n\n### Response: An independent study course involves one-on-one interactions between a student and a faculty member to undertake a learning opportunity otherwise unavailable. To be eligible, students must have completed more than half of the credit hours in their program of study with at least a 3.0 CGPA. However, they cannot register for independent study to make up deficiencies resulting from failures in other courses."},
    {"inputs": "### Instruction: What are the conditions for earning credits at other academic institutions?\n\n### Response: Students with good academic standing may enroll in courses at other institutions if the course is not offered in the current semester, conflicts with another required course, or if they are in their final graduation semester or term. The course must be equivalent to an ADU course, and the student's request will be evaluated by the College advisor and forwarded to the College Dean if conditions are met."},
    {"inputs": "### Instruction: What are the graduation requirements for undergraduate students?\n\n### Response: Undergraduate students must successfully complete all course requirements and other academic activities assigned to their specialized study plan. They must maintain a CGPA of at least 2.0 out of 4.0. To be eligible for graduation, students must complete the Application for Graduation Form online before the specified deadline for the semester or term."}

    ])

        # Fine-tuning the model
        num_epochs = 4
        for count in range(num_epochs):
            print(f"Fine-tuning the model, iteration {count + 1}")
            GRP2_newModel.fine_tune(samples=MyPrompts)

        def get_model_response(userQuery):
            MyNewOutput = GRP2_newModel.complete(query=userQuery).generated_output
            return MyNewOutput

        gr.Interface(fn=get_model_response,
                     outputs="text",
                     title="Fine-tuned Chatbot",
                     inputs="text",
                     description="This model has been fine-tuned with specific knowledge. Try asking something!").launch()

CreateBaseModel()

Created model adapter with id d1b5b300-c535-4b4b-911b-8c5ac510c2b6_model_adapter
[{'inputs': 'instruction what are administrative drops and under what circumstances are they initiated abu dhabi university response administrative drops abu dhabi university are initiated university officials for various reasons including failure meet prerequisites safety concerns academic disciplinary suspension disruptive behavior exceeding allowable absences course loads these drops ensure academic integrity and the safety students and faculty members its crucial for students aware these regulations and adhere them maintain their academic standing'}, {'inputs': 'instruction what the maximum number courses can take major semester abu dhabi university response the maximum number courses you can take major semester courses accumulating maximum credit hours'}, {'inputs': 'instruction what circumstances may result change students final grade abu dhabi university response there are several situations that ma

The Fine-tuned Model **after** Preprocessing

In [ ]:
def preprocess_text(text):
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    #Regex Processing
    text = re.sub(r'\b\w*\d+\w*\b', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = re.sub(r'\b\w*_\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def CreateBaseModel():
    with Gradient() as gradient:
        base_model = gradient.get_base_model(base_model_slug="nous-hermes2")
        Grp2_newModel = base_model.create_model_adapter(name="test model")

        print(f"Created model adapter with id {Grp2_newModel.id}")

        MyPrompts=([

        {"inputs": "### Instruction: What are administrative drops, and under what circumstances are they initiated in Abu Dhabi University?\n\n### Response: Administrative drops at Abu Dhabi University are initiated by university officials for various reasons, including failure to meet prerequisites, safety concerns, academic or disciplinary suspension, disruptive behavior, or exceeding allowable absences or course loads. These drops ensure academic integrity and the safety of students and faculty members. It's crucial for students to be aware of these regulations and adhere to them to maintain their academic standing."},
        {"inputs": "### Instruction: What is the maximum number of courses I can take in a major semester in Abu Dhabi University?\n\n### Response: The maximum number of courses you can take in a major semester is 6 courses, accumulating a maximum of 18 credit hours."},
        {"inputs": "### Instruction: What circumstances may result in a change of a student's final grade at Abu Dhabi University?\n\n### Response: There are several situations that may lead to a change in a student's final grade:\n\n- A grade appeal request by the student after an informal discussion with the faculty member.\n- Discovery of an error in calculating the student's final grade.\n- Change of an \"I\" (Incomplete) or \"IP\" (In Progress) grade to the actual grade upon release. It's crucial for students to be aware of these possibilities and the time limit for grade changes, which is one semester from the date the grades are posted by the Registrar."},
        {"inputs": "### Instruction: What does an advisor do in Abu Dhabi University?\n\n### Response: An advisor is a full-time staff member within each college who advises and counsels students on programs and course selection, institutional policies, career choices, effective study habits, and other academic and career-oriented decisions."},
        {"inputs": "### Instruction: What is the process for appealing a final grade at Abu Dhabi University?\n\n### Response: If a student wishes to appeal a final grade, they must follow the Grade Appeal Procedure outlined by the university. This involves a consultation process with the faculty member, department chairperson, and dean of the college offering the course. If the matter is not resolved, the student may initiate a Committee Appeal Process by filing a Grade Appeal Form with the Office of the Registrar. The committee reviews the student's performance and makes recommendations, with the final decision resting with the dean. It's important for students to adhere to the deadlines and procedures outlined for grade appeals."},
        {"inputs": "### Instruction: When is the add and drop period in Abu Dhabi University?\n\n### Response: A student is allowed to add/drop one or more courses during the first week of the regular semester and during the first two days of the Winter/Summer term."},
        {"inputs": "### Instruction: How are students classified based on their progression towards their Bachelor Degree at Abu Dhabi University?\n\n### Response: Each college at Abu Dhabi University has its own classification system based on the number of credit hours passed, tailored to the specific requirements of their programs. This classification helps students track their progress towards completing their Bachelor's Degree effectively:\n\n- College of Arts (CAS):\n  - First Year/Freshmen: 00 - 29 credit hours\n  - Second Year/Sophomores: 30 - 59 credit hours\n  - Third Year/Juniors: 60 - 89 credit hours\n  - Final Year/Seniors: 90+ credit hours\n\n- College of Business (COB):\n  - First Year/Freshmen: 00 - 30 credit hours\n  - Second Year/Sophomores: 31 - 60 credit hours\n  - Third Year/Juniors: 61 - 90 credit hours\n  - Final Year/Seniors: 91+ credit hours\n\n- College of Engineering (COE):\n  - Bachelor of Architecture\n    - First Year: 00 - 34 credit hours\n    - Second Year: 35 - 69 credit hours\n    - Third Year: 70 - 102 credit hours\n    - Fourth Year: 103 -132 credit hours\n    - Fifth Year: 133+ credit hours\n  - B.Sc. in Aviation, Biomedical Engineering, Chemical Engineering, Civil Engineering, Computer Engineering, Cybersecurity Engineering, Electrical Engineering, Industrial Engineering, Information Technology, Interior Design, Mechanical Engineering, and Software Engineering\n    - First Year/Freshmen: 00 - Various credit hours\n    - Second Year/Sophomores: Various - Various credit hours\n    - Third Year/Juniors: Various - Various credit hours\n    - Final Year/Seniors: Various+ credit hours\n\n- College of Health Sciences (CoHS):\n  - B.Sc. in Public Health, Environmental Health and Safety, Biomedical Sciences - Laboratory Medicine, Molecular and Medical Genetics, and Human Nutrition and Dietetics\n    - First Year/Freshmen: 00 - Various credit hours\n    - Second Year/Sophomores: Various - Various credit hours\n    - Third Year/Juniors: Various - Various credit hours\n    - Final Year/Seniors: Various+ credit hours"},
        {"inputs": "### Instruction: Can I drop courses after the add and drop period ends during the Spring/Fall semester in Abu Dhabi University?\n\n### Response: Students dropping courses within the first calendar week of the Fall/Spring semester will receive a 100% refund of the tuition fee. Students dropping courses in the second calendar week of the Fall/Spring semester will receive 75% refund of the tuition fee. In such cases, a “Withdrawal without Penalty” (W) grade will be entered in their records. Students dropping courses in the third calendar week of the Fall/Spring semester will receive a 50% refund of tuition fees. In such cases, a (W) grade will be entered in their records. Students dropping courses after the third week of the Fall/Spring semester will receive no refund, and will be awarded a (W) grade for that course. If students do not withdraw from courses during these specified periods, they will be considered as being registered for the courses and be held accountable."},
        {"inputs": "### Instruction: What are the consequences of violating the Academic Integrity Policy (AIP) at Abu Dhabi University?\n\n### Response: Violating the Academic Integrity Policy (AIP) at Abu Dhabi University can lead to severe consequences, including dismissal from the university. Examples of violations include cheating, plagiarism, fabrication of data, presenting false credentials, collusion, and free riding. Sanctions may include failing grades, course withdrawals, and ineligibility for tuition refunds."},
        {"inputs": "### Instruction: How can I apply for graduation in Abu Dhabi University?\n\n### Response: To apply for graduation, visit www.adu.ac.ae and log in to PeopleSoft using your username and password. Then, navigate to self-service and select 'degree progress/graduation.' Proceed to apply for graduation, choose your program, select the expected graduation term, and carefully read any instructions provided by the Office of the Registrar."},
        {"inputs": "### Instruction: What is the total credit hour requirement for the Cybersecurity Engineering program, and how is it distributed across different course categories in Abu Dhabi University?\n\n### Response: The Cybersecurity Engineering program at Abu Dhabi University requires 135 credit hours for completion. These are distributed across various categories, including general education (24 credit hours), college requirements (54 credit hours), major requirements (42 credit hours), major electives (6 credit hours), and open electives (3 credit hours)."},
        {"inputs": "### Instruction: How is the grading scale at ADU and what does it mean?\n\n### Response: At Abu Dhabi University, the grading scale ranges from A (4.00) to F (0.00). Each grade corresponds to a specific range of percentages and carries different meanings, such as excellent, very good, good, satisfactory, or fail. Additionally, there are other symbols like NP (Not Pass), SP (Special Pass), P (Pass), S (Successful Completion), U (Unsuccessful Completion), I (Incomplete), IP (In Progress), T (Transfer), H (Final Grade on Hold), W (Withdrawal from a Course), and WA (Withdrawal Due to Absence Limit)."},
        {"inputs": "### Instruction: What are the major requirements for the Bachelor of Science in Information Technology program at Abu Dhabi University?\n\n### Response: The major requirements for the Bachelor of Science in Information Technology program consist of 45 credit hours. These include courses such as Ethical Hacking, Data Warehousing and Data Mining, Web Design, Operating Systems, Artificial Intelligence, Software Engineering, and others. Each course has its prerequisites and contributes to building a comprehensive understanding of information technology."},
        {"inputs": "### Instruction: What are the major requirements for the Bachelor of Science in Cybersecurity Engineering program at Abu Dhabi University?\n\n### Response: The major requirements for the Bachelor of Science in Cybersecurity Engineering program include 42 credit hours. These courses cover essential topics such as Database Management Systems, Digital Logic Design, Web Design, Artificial Intelligence, Information Security, Internet of Things, Network Security, and Ethical Hacking. Prerequisites are specified for each course."},
        {"inputs": "### Instruction: Am I eligible to take my capstone next semester at Abu Dhabi University?\n\n### Response: To determine your eligibility for the capstone next semester, ensure you have reached the senior level (90 credit hours) and completed any prerequisites. The capstone typically requires a certain level of academic progress and completion of foundational coursework."},
        {"inputs": "### Instruction: Tell me about the Information Technology degree in Abu Dhabi University?\n\n### Response: The Information Technology (IT) program at Abu Dhabi University offers concentrations in Web Technology and Internet Applications, Networking and Security, and Mobile Applications and Interactive Media. These concentrations provide specialized knowledge and skills in areas such as web development, cybersecurity, mobile app development, and user interface design."},
        {"inputs": "### Instruction: How should I plan my course load for the next semester in Abu Dhabi University?\n\n### Response: When planning your course load for the next semester, consider the types of courses you intend to take and your ability to manage workload. It's advisable to balance major courses with electives and to avoid overloading yourself, especially if you anticipate difficulty with certain subjects."},
        {"inputs": "### Instruction: What are the major requirements for the Bachelor of Science in Software Engineering program at Abu Dhabi University?\n\n### Response: The major requirements for the Bachelor of Science in Software Engineering program consist of 36 credit hours. Courses cover topics such as Database Management Systems, Web Design, Artificial Intelligence, Distributed Information Systems, Information Security, and various software engineering principles."},
        {"inputs": "### Instruction: What are the major elective courses available in the Bachelor of Science in Software Engineering program at Abu Dhabi University?\n\n### Response: Major elective courses for the Bachelor of Science in Software Engineering program offer options such as Internet of Things, Data Science, Web Programming, E-Commerce Applications Design, and Selected Topics in Software Engineering. These courses allow students to tailor their education to specific interests within the field."},
        {"inputs": "### Instruction: What are the major elective courses available in the Bachelor of Science in Cybersecurity Engineering program at Abu Dhabi University?\n\n### Response: Major elective courses for the Bachelor of Science in Cybersecurity Engineering program include options like Data Science, System and Network Administration, Selected Topics in Cybersecurity, and various software engineering courses. These electives provide students with opportunities to specialize further in areas of interest."},
        {"inputs": "### Instruction: What are the major elective courses available in the Information Technology program at Abu Dhabi University?\n\n### Response: Major elective courses for the Information Technology program include options such as Computer Graphics and Animation, Mobile Device Security, Network Security and Forensics, Web Programming, and Advanced E-commerce Applications Design. These courses offer additional depth and specialization within the IT field."},
        {"inputs": "### Instruction: What concentrations are offered in the College of Engineering at Abu Dhabi University?\n\n### Response: The College of Engineering at Abu Dhabi University offers concentrations such as Artificial Intelligence in Computer Engineering, Robotics and Automation in Electrical Engineering, Industrial Mechatronics and Metallurgy in Mechanical Engineering, Water Technology in Chemical Engineering, and Cybersecurity and Game Development in Information Technology, among others."},
        {"inputs": "### Instruction: What are the requirements to enroll in the Engineering and Computer Science college Majors in Abu Dhabi University?\n\n### Response: To enroll in majors within the College of Engineering and Computer Science at Abu Dhabi University, you need to have completed a minimum of 24 semester credit hours of university requirements with a GPA of at least 2.25. Additionally, specific courses such as ITE 100, ENG 105, and MTT 101 must be completed with a grade of at least 2.0."},
        {"inputs": "### Instruction: How do I know which courses I should take next semester?\n\n### Response: To determine which courses to take next semester, refer to your study plan and identify the courses available in that semester. Some courses may only be offered in specific semesters, so it's essential to plan accordingly based on your program requirements and academic progress."},
        {"inputs": "### Instruction: Can I take free elective courses from any Major in the university?\n\n### Response: Yes, you can take free elective courses from any Major in the university, provided you meet the prerequisites. However, for the BEd Field Teaching (Arabic Language and Islamic Studies) curriculum, only 2 specific courses are allowed as electives."},
        {"inputs": "### Instruction: Do I have to complete a Minor?\n\n### Response: No, completing a Minor is optional but strongly encouraged. Minors consist of 6 courses (18 credits), which can be substituted for 6 elective courses in some Majors."},
        {"inputs": "### Instruction: What Majors and Minors are available?\n\n### Response: The list of Majors and minors is available in the Undergraduate/Postgraduate Catalog and the Advising website on the ADU portal. Minors offered are identical to the Majors available."},
        {"inputs": "### Instruction: When should I declare my Major?\n\n### Response: Decide your Major when you're close to completing the 14 University Requirement courses, as you'll then begin taking courses from your Major."},
        {"inputs": "### Instruction: What are the admission requirements for direct entry into the Information Technology program?\n\n### Response: Direct entry requires a minimum average of 65% for Elite Track, 70% for Advanced Track, and 75% for General Track or equivalent. EmSAT scores in Mathematics, Physics, and English Language are also required."},
        {"inputs": "### Instruction: How many undergraduate programs does the College of Engineering  offer, and what are they?\n\n### Response: The College offers thirteen undergraduate programs covering Mechanical, Industrial, Electrical, Computer, Biomedical, Cybersecurity, Software, Information Technology, Civil, Chemical, Architecture, Interior Design, and Aviation."},
        {"inputs": "### Instruction: What are the different scholarships the university offers?\n\n### Response: Scholarships include H.H. Sheikh Hamdan Bin Zayed Scholarship, Chairman’s Scholarship, University Scholarship, Academic Scholarship, and Athletic Scholarship."},
        {"inputs": "### Instruction: How can I achieve Latin honors at ADU?\n\n### Response: To achieve Latin honors, maintain a Cumulative Grade Point Average (CGPA) of 3.5 or above upon graduation, resulting in titles like Cum laude (3.50-3.69), Magna cum laude (3.70-3.89), or Summa cum laude (3.90-4.00)."},
        {"inputs": "### Instruction: What are the requirements to apply for financial aid?\n\n### Response: Financial aid is determined based on eligibility and need, ranging from 10% to 30% tuition fee waiver. Initial requirements include UAE permanent residency, financial need evidence, and meeting English proficiency standards."},
        {"inputs": "### Instruction: If I Withdraw From The University, Can I Be Re-Admitted?\n\n### Response: Former students in good academic standing, who withdrew for reasons like financial issues or medical conditions, may seek re-admission. They must submit a petition to the Registrar at least two months before the semester begins."},
        {"inputs": "### Instruction: Can I Change My Major?\n\n### Response: Yes, you can change your Major at any time during the semester if in good standing. Simply fill out a Change Major Application Form at the college of the newly chosen major."},
        {"inputs": "### Instruction: What do I do if there is no more seats in a course I want to register in?\n\n### Response: You can talk to your advisor to either put you on a waiting list or talk to the department head to open another section. This can be done if a good number of students also need to take this course. If that's not possible, you will have to wait until the next semester/year."},
        {"inputs": "### Instruction: What is the Software Engineering degree about?\n\n### Response: The Bachelor of Science in Software Engineering program at Abu Dhabi University is designed to cultivate a new generation of software engineers equipped to tackle the multifaceted challenges of contemporary software development. Rooted in the application of engineering principles to the systematic creation of software, this program provides students with a comprehensive understanding of the theoretical underpinnings and practical applications essential for success in the field. Central to the program's mission is the development of technical proficiency, problem-solving abilities, and leadership qualities among its graduates. Through rigorous coursework and hands-on projects, students gain proficiency in key areas such as software maintenance and evolution, requirements engineering, software design and architecture, testing and quality assurance, web design, and more. By engaging with real-world challenges and industry-standard tools, students acquire the skills and confidence to navigate the complexities of modern software development projects. Moreover, the program fosters a culture of lifelong learning, recognizing the dynamic nature of the software industry and the importance of continuous skill development. Students are encouraged to pursue training, continuing education, and even graduate studies to stay abreast of emerging technologies and industry trends. By instilling a commitment to ongoing learning and professional growth, the program prepares graduates to adapt to evolving industry demands and thrive in their careers over the long term. Beyond technical skills, the program also places a strong emphasis on ethical values and social responsibility. Students are encouraged to consider the broader implications of their work on economic, environmental, social, political, and ethical dimensions. By upholding ethical standards and contributing to the development and growth of local and global communities, graduates of the program become not only proficient software engineers but also conscientious global citizens. In essence, the Bachelor of Science in Software Engineering program at Abu Dhabi University offers a holistic educational experience that prepares students to excel in the dynamic and ever-evolving field of software development. By fostering a blend of technical expertise, problem-solving abilities, lifelong learning habits, and ethical values, the program equips graduates with the tools they need to make meaningful contributions to both the software industry and society at large, positioning them for success in their careers and beyond."},
        {"inputs": "### Instruction: Can I drop courses after the add and drop period ends during the Winter/Summer semester?\n\n### Response: Students dropping courses within the first two days of the calendar days of the Winter/Summer semester will receive a 100% refund of the tuition fee. Students dropping courses in the third and fourth calendar days of the Winter/Summer semester will receive 75% refund of the tuition fee. In such cases, a â€œWithdrawal without Penaltyâ€ (W) grade will be entered in their records. Students dropping courses in the first and second days of the next calendar week of the Winter/Summer semester will receive 50% refund of tuition fees. In such cases, a (W) grade will be entered in their records. Students dropping courses after the two days of the second week will receive no refund, and will be awarded a (W) grade for that course. If students do not withdraw from courses during these specified periods, they will be considered as being registered for the courses and be held accountable. A 100% refund of tuition fees will be given for courses cancelled by Abu Dhabi University."},
        {"inputs": "### Instruction: What career paths are available with a degree in CSIST?\n\n### Response: There are multiple career paths in the computer science and information technology field. It all depends on your goals, aspirations, and preferences."},
        {"inputs": "### Instruction: What are the minimum admission requirements for direct entry into engineering programs?\n\n### Response: For direct admission to engineering programs, applicants must have a minimum high school average of 75% for the Elite Track or 80% for the Advanced Track, and 90% for the General Track or equivalent in standardized international systems. Additionally, they must pass EmSAT in Mathematics and Physics with scores of 800 or equivalent, and EmSAT in English Language with a score of 1100, or an approved equivalent English Proficiency test."},
        {"inputs": "### Instruction: How can I apply for internships?\n\n### Response: First, you need to make sure that you have completed the required number of credit hours. Then you can apply for internship on ADU groups once the portal opens. Then you need to start looking for companies or inform the department that you need help in looking for companies and they will guide you accordingly."},
        {"inputs": "### Instruction: What is the purpose of the grades P (credit), P (non-credit), S, and U?\n\n### Response: These grades indicate the completion status of certain academic activities: P (credit): Satisfactory completion of credit-bearing coursework P (non-credit): Satisfactory completion of non-credit ELI or undergraduate courses/Internship S: Satisfactory completion of graduate courses U: Unsatisfactory completion of graduate courses"},
        {"inputs": "### Instruction: What should I do if a required course is not available next semester?\n\n### Response: Inform your academic advisor then your department head so they can open the course."},
        {"inputs": "### Instruction: Do all students have to take a math placement test?\n\n### Response: Yes, all College of Engineering students must take the Math Placement Test (MPT) administered by the University College. Based on the MPT result, students will be placed in one of the math courses (MTH 100, MTT 101, or MTT 102) depending on their program of study."},
        {"inputs": "### Instruction: What do the grade notations NP, SP, P, S, U, I, IP, T, W, WA, and H stand for in Abu Dhabi University's grading system?\n\n### Response: These notations represent different aspects of a student's academic performance or status: NP: Not Pass SP: Special Pass P: Pass S: Successful Completion U: Unsuccessful Completion I: Incomplete IP: In Progress T: Transfer W: Withdrawal from a Course WA: Withdrawal Due to Absence Limit H: Final Grade on Hold"},
        {"inputs": "### Instruction: What additional admission requirement applies to applicants to the Bachelor of Architecture program?\n\n### Response: Applicants to the Bachelor of Architecture Program are required to submit or present a portfolio of graphic work for evaluation as part of the admission requirements. The portfolio should demonstrate creativity and/or artistic skill. Applicants may also be selectively interviewed by two members of the teaching staff to assess their genuine interest in the subject and evidence of creative intent."},
        {"inputs": "### Instruction: What is the grading scale used for undergraduate courses at Abu Dhabi University?\n\n### Response: Undergraduate courses at Abu Dhabi University use the following grading scale: A: 4.00 (90-100) Excellent B+: 3.50 (85-89) Very Good B: 3.00 (80-84) Very Good C+: 2.50 (75-79) Good C: 2.00 (70-74) Good D+: 1.50 (65-69) Satisfactory D: 1.00 (60-64) Satisfactory F: 0.00 (Less than 60) Fail"},
        {"inputs": "### Instruction: What is an independent study course, and who is eligible to participate?\n\n### Response: An independent study course involves one-on-one interactions between a student and a faculty member to undertake a learning opportunity otherwise unavailable. To be eligible, students must have completed more than half of the credit hours in their program of study with at least a 3.0 CGPA. However, they cannot register for independent study to make up deficiencies resulting from failures in other courses."},
        {"inputs": "### Instruction: What are the conditions for earning credits at other academic institutions?\n\n### Response: Students with good academic standing may enroll in courses at other institutions if the course is not offered in the current semester, conflicts with another required course, or if they are in their final graduation semester or term. The course must be equivalent to an ADU course, and the student's request will be evaluated by the College advisor and forwarded to the College Dean if conditions are met."},
        {"inputs": "### Instruction: What are the graduation requirements for undergraduate students?\n\n### Response: Undergraduate students must successfully complete all course requirements and other academic activities assigned to their specialized study plan. They must maintain a CGPA of at least 2.0 out of 4.0. To be eligible for graduation, students must complete the Application for Graduation Form online before the specified deadline for the semester or term."}

        ])

        for prompt in MyPrompts:
            prompt["inputs"] = preprocess_text(prompt["inputs"])
        print(MyPrompts)


        # Fine-tuning the model
        num_epochs = 5
        for count in range(num_epochs):
            print(f"Fine-tuning the model, iteration {count + 1}")
            Grp2_newModel.fine_tune(samples = MyPrompts)

        def get_model_response(userQuery):
            MyNewOutput = Grp2_newModel.complete(query = preprocess_text(userQuery)).generated_output
            return MyNewOutput

        gr.Interface(fn=get_model_response,
                     outputs="text",
                     title="Fine-tuned Chatbot",
                     inputs="text",
                     description="This model has been fine-tuned with specific knowledge. Try asking something!").launch()

CreateBaseModel()


Created model adapter with id 2ade12e8-aeb4-4010-96c5-305c77c3d7a8_model_adapter
[{'inputs': 'instruction what are administrative drops and under what circumstances are they initiated abu dhabi university response administrative drops abu dhabi university are initiated university officials for various reasons including failure meet prerequisites safety concerns academic disciplinary suspension disruptive behavior exceeding allowable absences course loads these drops ensure academic integrity and the safety students and faculty members its crucial for students aware these regulations and adhere them maintain their academic standing'}, {'inputs': 'instruction what the maximum number courses can take major semester abu dhabi university response the maximum number courses you can take major semester courses accumulating maximum credit hours'}, {'inputs': 'instruction what circumstances may result change students final grade abu dhabi university response there are several situations that ma